In [1]:
import tensorflow as tf
import numpy as np
import scipy
from scipy import stats
from mt_layers import SplineWeight1D
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import h5py
from tqdm import tqdm

In [2]:
f = h5py.File('./data/MT_Splice/psi_data_logit.h5','r')
seql_test = f['seql_test']
seqr_test = f['seqr_test']
y_test = f['y_test']
mean_test = y_test[:,:,1]
y_test = y_test[:,:,0]


In [3]:
express_mask = np.sum(~np.isnan(y_test),axis=1) >= 10
m_psi = tf.sigmoid(mean_test)
y_psi = tf.sigmoid(y_test)
diff_psi = np.abs(m_psi - y_psi)
diff_mask = np.sum(diff_psi>= 0.2,axis = 1) >=1
mask = express_mask * diff_mask

2022-08-02 11:13:13.505481: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-08-02 11:13:13.594523: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:21:00.0 name: NVIDIA RTX A4000 computeCapability: 8.6
coreClock: 1.56GHz coreCount: 48 deviceMemorySize: 15.74GiB deviceMemoryBandwidth: 417.29GiB/s
2022-08-02 11:13:13.594721: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64
2022-08-02 11:13:13.594765: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcublas.so.10'; dlerror: libcublas.so.10: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64
2022-08-02 11:13:13.595912: I t

In [4]:
average_y = []
for i in range(0,4):
    model = tf.keras.models.load_model('./replications/MMSplice_MTSplice/mmsplice/models/mtsplice_deep'+str(i)+'.h5',
                                        custom_objects={'SplineWeight1D':SplineWeight1D})
    y_pred = model.predict([seql_test[mask,:,:],seqr_test[mask,:,:]])
    average_y.append(y_pred)

In [5]:
mean_y = np.mean(average_y,axis = 0)
mean_y.shape

(1621, 56)

In [6]:
selected_y = y_test[mask,:]
sum = []
for i in tqdm(range(0,56)):
    corr,_ = scipy.stats.spearmanr(selected_y[:,i],mean_y,nan_policy='omit')
    sum.append(corr)

  0%|          | 0/56 [00:00<?, ?it/s]


NameError: name 'selected_y' is not defined

In [2]:
import h5py
import math
import numpy as np
import sys
sys.path.append('./data_generation/')
from data_preprocess import logit
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [3]:
perf_f = h5py.File('./data/MT_Splice/test_ascot_performance.h5','r')
perf_f.keys()

<KeysViewHDF5 ['measured', 'pred', 'x', 'x_hat']>

In [20]:
logit(perf_f['measured'][0]) - np.nanmean(logit(perf_f['measured'][0]))

array([        nan,         nan,  0.3757556 ,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan, -0.29421248, -0.78462315, -0.50781025,
               nan,         nan,  8.39480203,         nan,         nan,
               nan,         nan,         nan,         nan,  0.37379871,
               nan,         nan,         nan,         nan,         nan,
               nan, -0.37825141,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,  0.27191064, -0.05920082,
               nan,  0.58028318,  0.01791421,         nan,         nan,
               nan,  0.27414853,         nan,         nan,         nan,
               nan])

In [17]:
perf_f['x'][0]

array([        nan,         nan, -0.31295397,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan, -0.98292205, -1.47333272, -1.19651981,
               nan,         nan,  7.70609246,         nan,         nan,
               nan,         nan,         nan,         nan, -0.31491086,
               nan,         nan,         nan,         nan,         nan,
               nan, -1.06696097,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan, -0.41679892, -0.74791039,
               nan, -0.10842639, -0.67079535,         nan,         nan,
               nan, -0.41456103,         nan,         nan,         nan,
               nan])